In [1]:
%pwd 

'/home/verge/Desktop/docker_test/backend/notebook'

In [2]:
%cd .. 

/home/verge/Desktop/docker_test/backend


/home/verge/miniconda3/envs/rag/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableMap
from langchain_core.documents import Document

from IPython.display import Markdown, display

In [4]:
from langchain_ollama import ChatOllama, OllamaEmbeddings


ollama_llm = ChatOllama(
    model= 'llama3.2-vision',
    base_url="http://localhost:11434",
    temperature=0.7,
    num_ctx = 1024, # size of the context window
    num_predict = -2, # Maximum number of tokens to predict when generating text.
    top_k=40,
    top_p=0.9,  
)

ollama_embedding = OllamaEmbeddings(
    model="llama3.2-vision",
    base_url="http://localhost:11434"
)

In [5]:
# for directory loader 


loader = PyPDFLoader('data/B13 - Chapter II - Procedure  for proceeding Abroad.pdf')
documents = loader.load()
len(documents)

11

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

chunks = text_splitter.split_documents(documents)
len(chunks)

15

In [7]:
from src.vectorstore_milvus import MilvusVectorStoreManager

vector = MilvusVectorStoreManager(embeddings=ollama_embedding, collection_name='test_collection')

# vector.delete_collection()

2025-01-08 12:29:05 - INFO - vectorstore_milvus.py:201 - Connected to Milvus at {'uri': 'database/milvushr.db'}


In [8]:
vector.print_collection_summary()

2025-01-08 12:29:05 - INFO - vectorstore_milvus.py:272 - Generating Milvus Collection Summary
Milvus Collection Summary:
------------------------------
Total Collections: 1
Collection Name : test_collection ----- No. of Records : 90
------------------------------
2025-01-08 12:29:05 - INFO - vectorstore_milvus.py:284 - Successfully printed collection summary


In [9]:
vec = vector.create_collection(documents=chunks)

2025-01-08 12:29:06 - INFO - vectorstore_milvus.py:222 - Successfully created new collection test_collection with 15 of documents


In [10]:
vec1 = vector.load_existing_vector_store()

2025-01-08 12:29:06 - INFO - vectorstore_milvus.py:243 - Successfully loaded existing vector store: test_collection with 105 records


In [11]:
vector.add_documents(documents=chunks, vector_store=vec)

2025-01-08 12:29:07 - INFO - vectorstore_milvus.py:260 - Successfully added 15 documents to vector store


[455165751507812382,
 455165751507812383,
 455165751507812384,
 455165751507812385,
 455165751507812386,
 455165751507812387,
 455165751507812388,
 455165751507812389,
 455165751507812390,
 455165751507812391,
 455165751507812392,
 455165751507812393,
 455165751507812394,
 455165751507812395,
 455165751507812396]

In [12]:
vec.as_retriever().invoke('what is passport')

[Document(metadata={'page': 4, 'pk': 455165437052190726, 'source': 'data/B13 - Chapter II - Procedure  for proceeding Abroad.pdf'}, page_content='HR - Executives’ Manual Page B 13 / 5a CHAPTER -II\nPOLICIES, PROCEDURES & RULES\n6-13: Procedure for proceeding Abroad\nISSUE OF NO OBJECTION CERTIFICATE FOR PASSPORT ONLY\nCertified that Shri ______________________ son of Shri ________________\nis a permanent employee of Mazagon Dock Shipbuilders Limited, Dockyard\nRoad, Mazagon, Mumbai 400 010 from ________ (date) and is presently\nholding the post of _________________ and his identity is certified. This\norganization has No Objection to his acquiring Indian Passport. The\nundersigned is duly authorized to sign this Identity Certificate. I have read the\nprovisions of Section 6(2) of the Passports Act, 1967 and certify that these are\nnot attracted in the case of this applicant. I recommend issue of an Indian\nPassport to him. It is certified that this organization is a Central Public Sect

In [13]:
vec1.as_retriever().invoke('what is passport')

[Document(metadata={'page': 4, 'pk': 455165437052190726, 'source': 'data/B13 - Chapter II - Procedure  for proceeding Abroad.pdf'}, page_content='HR - Executives’ Manual Page B 13 / 5a CHAPTER -II\nPOLICIES, PROCEDURES & RULES\n6-13: Procedure for proceeding Abroad\nISSUE OF NO OBJECTION CERTIFICATE FOR PASSPORT ONLY\nCertified that Shri ______________________ son of Shri ________________\nis a permanent employee of Mazagon Dock Shipbuilders Limited, Dockyard\nRoad, Mazagon, Mumbai 400 010 from ________ (date) and is presently\nholding the post of _________________ and his identity is certified. This\norganization has No Objection to his acquiring Indian Passport. The\nundersigned is duly authorized to sign this Identity Certificate. I have read the\nprovisions of Section 6(2) of the Passports Act, 1967 and certify that these are\nnot attracted in the case of this applicant. I recommend issue of an Indian\nPassport to him. It is certified that this organization is a Central Public Sect

In [14]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(
   "database/milvushr.db"
)

res = client.list_collections()

print(res)

['test_collection']


In [15]:
client.get_collection_stats('test_collection')


{'row_count': 120}

In [21]:
if client.has_collection('test_collection'):
    client.colse()
    print('connection closed')

ConnectionNotExistException: <ConnectionNotExistException: (code=1, message=should create connection first.)>

In [17]:
collections = client.list_collections()
print(f"Total No. of Collections: {len(collections)}")

for name in collections:
    print(f"Collection Name : {name} ----- No. of Records : {client.get_collection_stats(name)['row_count']}")


Total No. of Collections: 1
Collection Name : test_collection ----- No. of Records : 120


In [22]:
client.close()